# BOFdat step 1

## Generate stoichiometric coefficients for the major macromolecules of the cell and calculate maintenance cost

In [14]:
from BOFdat import step1
from BOFdat.util import update
import pandas as pd
import cobra

## Example using the *E.coli* genome-scale model *i*ML1515

The weight percentage and abundance of each molecule in the cell may vary from an organism to another and vary between growth conditions for a given organism [1,2]. BOFdat allows to incorporate macromolecular cell composition obtained from literature or new experiments to generate new stoichiometric coefficients for your model's biomass objective function (BOF). Once weight percentages are obtained, OMIC data can be incorporated to buff the coefficients and fit to experimental reality. 

### Steps

The following example will lead you through all the necessary steps for the generation of the BOF stoichiometric coefficients (BOFsc) for *E.coli* K12 MG1655 GEM *i*ML1515 [3]. 

1. Obtain the macromolecular composition of the organism

2. Obtain OMICs experimental data

3. Generate BOFsc

4. Generate NGAM and GAM

5. Update BOF (BOFdat!)


### Sources

[1]  Dennis P. Patrick and Bremmer Hans. (1974) Macromolecular composition during steady-state growth of *Escherichia coli* B/r. Journal of bacteriology


[2] Benjamin Volkmer and Matthias Heinemann. (2011) Condition-Dependent Cell Volume and Concentration of Escherichia coli to Facilitate Data Conversion for Systems Biology Modeling. PLoS One


[3] Jonathan M Monk, Colton J Lloyd, Elizabeth Brunk, Nathan Mih, Anand Sastry, Zachary King, Rikiya Takeuchi, Wataru Nomura, Zhen Zhang, Hirotada Mori, Adam M Feist and Bernhard O Palsson. (2017) *i*ML1515, a knowledgebase that computes Escherichia coli traits. Nat. Biotech.

## Obtain the macromolecular compositon of the organism

*E.coli* has been characterized thoroughly in literature. The BOFsc used in *i*AF1260 [4] are the same in *i*ML1515 [3] and were obtained from Neidhart *et. al* [5].

**Note:** The package also provides the option to include the percentage of each type of RNA molecule in the cell (ribosomal, transfer and messenger). The default values are rRNA: 0.9, tRNA 0.05 and mRNA: 0.05.

### Sources

[4] Adam M Feist, Christopher S Henry, Jennifer L Reed, Markus Krummenacker, Andrew R Joyce, Peter D Karp, Linda J Broadbelt, Vassily Hatzimanikatis and Bernhard Ø Palsson. (2007) A genome-scale metabolic reconstruction for Escherichia coli K-12 MG1655 that accounts for 1260 ORFs and thermodynamic information. Mol. Syst. Bio. 

[5] Neidhardt FC, Ingraham JL, Schaechter M (1990) Physiology of the Bacterial Cell: a Molecular Approach. Sinauer Associates: Sunderland, Mass

In [15]:
#Set parameters based on dry weight composition
dna_weight_fraction = 0.038
rna_weight_fraction = 0.09
protein_weight_fraction = 0.585
lipid_weight_fraction = 0.019
metabolite_weight_fraction = 0.22

## Inputs

### DNA
    
    - Genome sequence (FASTA DNA)
    - DNA macromolecular weight fraction 
    - Model (JSON or SBML)
    
    
### RNA 

    - Genome annotation (GenBank)
    - Transcriptomic data (2 column CSV)
    - RNA macromolecular weight fraction 
    - Model (JSON or SBML)
    
    
### Proteins

    - Genome annotation (GenBank)
    - Proteomic data (2 column CSV)
    - Protein macromolecular weight fraction
    - Model (JSON or SBML)
    
    
### Lipids

    - Lipidomic abundances (2 column CSV)
    - Lipidomic conversion to model identifiers (2 column CSV)
    - Lipid macromolecular weight fraction
    - Model (JSON or SBML)
    

### Maintenance costs

    - Growth and uptake rates on different conditions (CSV)

In [21]:
#Give the path to each file as function parameters
#Genome file in BioPython supported format (.faa, .fna) and GenBank file 
#also in BioPython supported format (.gb, .gbff)
genome = '/home/ana/DynamicBiomass/dynamicBOF/syneco/psyneco_genome.fna'
#genbank = '/home/ana/DynamicBiomass/dynamicBOF/E.coliML1515GENOME/iML1515Genbank.gbff'
genbank= '/home/ana/DynamicBiomass/dynamicBOF/syneco/genomic_annotation.gbff'
#Omic datasets as a 2 column csv file, gene and abundance
transcriptomic = '/home/ana/DynamicBiomass/dynamicBOF/syneco/transcriptomic.csv'
proteomic = '/home/ana/DynamicBiomass/dynamicBOF/syneco/proteomic.csv'

#Lipidomic abundances and conversion to model identifier
lipidomic_abundances = '/home/ana/DynamicBiomass/dynamicBOF/syneco/lipidomic_abundances.csv'

lipidomic_conversion = '/home/ana/DynamicBiomass/dynamicBOF/syneco/lipidomic_conversion.csv'


#Growth data on different carbon sources, uptake and secretion rates
maintenance = 'data/maintenance.csv'

#The model for which the coefficients are generated
model = '/home/ana/DynamicBiomass/dynamicBOF/syneco/syneco_acetate.xml'
#model= 'data/iML1515.json'

## Obtain OMICs experimental data

Your genome should have a GenBank annotated file. This file should be provided in a BioPython supported format (.gb, .gbff). 

Search in literature allowed to find multiple OMICs dataset for different macromolecules that can be used to generate stoichiometric coefficients [6,7,8]. The data should be converted into a 2 column csv file. 
The genome file should be provided in a standard BioPython supported format (.faa or .fna) and is used to calculate the abundance of each base in the genome.

Transcriptomic and proteomic files are 2 column csv files where the first column is the **gene identifier ** and the second column is the relative abundance of each of these genes in the cell. 

Unlike DNA, RNA and proteins that are standard amongst every known life form, the lipid and metabolites in different organisms may vary. Hence a conversion file is required. This first column of this file is the original name of the compound and the second is the target identifier that this compound should have in your model. The first column of the abundance file gives the compound identifier in the model and the second column gives the abundance of that compound in the OMIC dataset.

### Sources

[6] Sang Woo Seo, Donghyuk Kim, Haythem Latif, Edward J. O’Brien, Richard Szubin & Bernhard O. Palsson. (2014) Deciphering Fur transcriptional regulatory network highlights its complex role beyond iron metabolism in Escherichia coli. Nat. Comm. 


[7] Alexander Schmidt, Karl Kochanowski, Silke Vedelaar, Erik Ahrné, Benjamin Volkmer, Luciano Callipo, Kèvin Knoops, Manuel Bauer, Ruedi Aebersold and Matthias Heinemann. (2016) The quantitative and condition-dependent *Escherichia coli* proteome. Nat. Biotech. 


[8] Kian-Kai Cheng, Baek-Seok Lee, Takeshi Masuda, Takuro Ito, Kazutaka Ikeda, Akiyoshi Hirayama, Lingli Deng, Jiyang Dong, Kazuyuki Shimizu, Tomoyoshi Soga, Masaru Tomita, Bernhard O. Palsson and Martin Robert. (2014) Global metabolic network reorganization by adaptive mutations allows fast growth of Escherichia coli on glycerol. Nat Comm.

## Generate BOFsc for macromolecules and generate maintenance costs

BOFdat operates with a single get_coefficient function for each macromolecule used. Input the parameters determined above as function parameters. Each function outputs a dictionary of metabolite and stoichiometric coefficients. The dictionary can be used to update the BOF (Step 5).

In [17]:
dna_coefficients = step1.generate_dna_coefficients(genome,model,DNA_WEIGHT_FRACTION=dna_weight_fraction)
dna_coefficients

/home/ana/anaconda3/lib/python3.9/site-packages/BOFdat/core/dna.py:19: UserWarning: 3 handles in the genome file.This may indicate that your genome is not completely assembled. 
BOFdat will parse the contigs but the stoichiometric coefficients may not be accurate.
  warnings.warn('%s handles in the genome file.This may indicate that your genome is not completely assembled. \nBOFdat will parse the contigs but the stoichiometric coefficients may not be accurate.'%(len(genome),))


{<Metabolite datp_c at 0x7f587ed376d0>: -0.011062168714647816,
 <Metabolite dttp_c at 0x7f587e1fffd0>: -0.011391034686741281,
 <Metabolite dctp_c at 0x7f587ed1c910>: -0.014909075347739362,
 <Metabolite dgtp_c at 0x7f587ed37490>: -0.013090897537980387,
 <Metabolite ppi_c at 0x7f587ed37070>: 0.050453176287108846}

In [22]:
rna_coefficients = step1.generate_rna_coefficients(genbank,model,transcriptomic,RNA_WEIGHT_FRACTION=rna_weight_fraction)
rna_coefficients

imported file          identifiers  abundances
0    Synpcc7942_1448       4.819
1    Synpcc7942_1029      15.055
2    Synpcc7942_2047      34.264
3    Synpcc7942_2046     185.108
4    Synpcc7942_2308      59.313
..               ...         ...
493  Synpcc7942_1255       7.754
494  Synpcc7942_0983      54.680
495  Synpcc7942_1573      48.886
496  Synpcc7942_1019     589.077
497  Synpcc7942_1085      54.565

[498 rows x 2 columns]
merged dataframes                 locus                                           sequence  \
0     Synpcc7942_0001  AUGCUUUGGCAAGAUUGCGAUCAAAGGCUCGGGCAGCCUCCCCCCA...   
1     Synpcc7942_0003  AUGACGGCGAUCUCCUCUGCUCCUUUUUCGGCCGAUGAAAUUGCUG...   
2     Synpcc7942_0004  AUGAUCCCGACUCAGCCGCUGACCGCCGACCUGGAUUGCGAUUUGG...   
3     Synpcc7942_0009  AUGGGUCGCGCCAAGCGGGUGGUUCUGGCCUAUUCAGGAGGCGUCG...   
4     Synpcc7942_0018  AUGCCGGCCUUUCUACUAGAAGUCGGAACCGAAGAACUCCCUGCUU...   
..                ...                                                ...   
492   Synpcc7942

/home/ana/anaconda3/lib/python3.9/site-packages/BOFdat/core/rna.py:60: UserWarning: Some identifiers not found in provided annotation
  warnings.warn('Some identifiers not found in provided annotation')


{<Metabolite atp_c at 0x7f587e97a160>: -0.05727676869912993,
 <Metabolite utp_c at 0x7f587cc04520>: -0.06750279891604867,
 <Metabolite ctp_c at 0x7f587e97c8b0>: -0.0820677336158242,
 <Metabolite gtp_c at 0x7f587cc04a60>: -0.07449226015625217,
 <Metabolite ppi_c at 0x7f587e97a7f0>: 0.28133956138725497}

In [18]:
protein_coefficients = step1.generate_protein_coefficients(genbank,model,proteomic,PROTEIN_WEIGHT_FRACTION=protein_weight_fraction)
protein_coefficients

{<Metabolite ala__L_c at 0x7f587e1230d0>: -0.8714798733118607,
 <Metabolite cys__L_c at 0x7f587e12ec40>: -0.061399099413228304,
 <Metabolite asp__L_c at 0x7f587e101190>: -0.26355234685189144,
 <Metabolite glu__L_c at 0x7f587e135520>: -0.2631937247039942,
 <Metabolite phe__L_c at 0x7f587e20f4f0>: -0.13767031725421291,
 <Metabolite gly_c at 0x7f587e213310>: -0.7350376418043463,
 <Metabolite his__L_c at 0x7f587e10f820>: -0.07417189127459949,
 <Metabolite ile__L_c at 0x7f587e21ae50>: -0.2811648004726669,
 <Metabolite lys__L_c at 0x7f587e101b80>: -0.11807797388913908,
 <Metabolite leu__L_c at 0x7f587e10a6a0>: -0.6354665767769733,
 <Metabolite met__L_c at 0x7f587e20ecd0>: -0.06960172462121227,
 <Metabolite asn__L_c at 0x7f587e20e310>: -0.13890756235221532,
 <Metabolite pro__L_c at 0x7f587e13b250>: -0.3300852662658746,
 <Metabolite gln__L_c at 0x7f587e13b5b0>: -0.2811219957159562,
 <Metabolite arg__L_c at 0x7f587e20faf0>: -0.2529905845835297,
 <Metabolite ser__L_c at 0x7f587e207250>: -0.40072

In [19]:
lipid_coefficients = step1.generate_lipid_coefficients(lipidomic_abundances,lipidomic_conversion,model,LIPID_WEIGHT_FRACTION=lipid_weight_fraction)
lipid_coefficients

{<Metabolite caro_c at 0x7f587d97d790>: -0.0023112075191766904,
 <Metabolite cholphya_c at 0x7f587d97dee0>: -0.006950081222152522,
 <Metabolite hpdcn_c at 0x7f587daf4b50>: -0.025823925413438868,
 <Metabolite phycy_c at 0x7f587d3d7a00>: -0.004884847851718013,
 <Metabolite zeax_c at 0x7f587d3d7730>: -0.0043657462481074515}

## Generate GAM and NGAM

Growth-associated maintenance (GAM) is the ATP cost related to growth. This includes the polymerization cost of each macromolecule. This cost is unaccounted for in the BOF because the model synthesizes the building blocks of each macromolecule in sufficient quantity to reflect the cell composition but not the cost of assembling those building blocks together. The GAM can be calculated experimentally by growing the bacteria on different sources of carbon at different starting concentrations. The carbon source should be the sole source of carbon in the media and its concentration should be measured after a given time. These remaining concentrations along with the excretion products are used by the package to constrain the model and calculate the ATP cost of growth. 


In [24]:
maintenance_cost = step1.generate_maintenance_costs(maintenance,model)

FileNotFoundError: [Errno 2] No such file or directory: 'data/maintenance.csv'

## Update BOF (BOFdat!)

All the dictionaries have been generated. Now it would be fun to start playing with the model. Actually BOFdat allows you to use the generated dictionaries to update and buff your BOF experimental data. Just buff that!

In [25]:
json_model = cobra.io.read_sbml_model(model)
bofdat_step1 = update.make_new_BOF(json_model,False,True,dna_coefficients,rna_coefficients,protein_coefficients,
                    lipid_coefficients,maintenance=None)

KeyError: 'Maintenance costs not provided'

In [10]:
#Save the step1 objective function for use in step2
bofdat_step1.to_csv('/home/ana/DynamicBiomass/dynamicBOF/bofdat_iML1515_Glc_Paper_step1.csv')

In [7]:
def _get_protein_sequence(path_to_genbank):
    # Get the prot id and sequence of each protein from genbank file
    from Bio import SeqIO
    genome_record = SeqIO.parse(path_to_genbank, 'genbank')
    seq_dict = {}
    for record in genome_record:
        for i in record.features:
            if i.type == 'CDS' and 'protein_id' in i.qualifiers:
                seq_dict[i.qualifiers['protein_id'][0]] = i.qualifiers['translation'][0]

    return seq_dict

In [10]:
_get_protein_sequence(genbank)

{'WP_070105208.1': 'MKLVCRQNELNTSLSLVSRAVPSRPNHPVLANVLLAADAGTQRLSLTAFDLSLGIQTSFAAQVERSGAITLPAKLLNDIVSRLPNDSDVTLEDNDAAATLSVGSGQYQMRGISADEFPELPLVQSQEALQLSASALIEGLRGTLFATSGDETKQILTGVHLKVQPDGLEFAATDGHRLAVVKTENAAATPATEFAVTVPSRALRDLERMIAIRGSDEAIALYHDQGQTVFQWGDQYLTSRTLDGQYPNYGQLIPREFNRNVAVDRKRLLAALERIAVLADQQNNAIRLSLDPENNRLALAVDAQDVGSGQEAVPAEIIGEPLEIAFNVRYLAEGLKALNTTDIQIQLNSNTSPVVLSPLGPVKITYLVMPIQLRS',
 'WP_199290672.1': 'MSDQPEELRVSDLLNRQVLDRQTTDEHGRIDRVWMHPPAHRVLGFLCKQGLIRGEFSAFRLDQLHALGDDSVVLEAAPQPANPEKVDRLESLLQHEVWTDAGLHVGKIVDCRFDRQTGYISAYLVSPHGWRGVAGMLWDLDPQLVLSYGQARVLVAELDPETLPVYEAGLTEAVAGSEPDYRHLLGDLRQRARHWKQELKLQVSKVKEQAQEVLAQVQEVREQVQQPLDWEDATPQPLTAIAEDEEDWDAAETALQPLPELRPKSAAIAPPAQPLIPPEAWDDDDPWV',
 'WP_011243804.1': 'MTAISSAPFSADEIAGEGIKPEEYDAIVERLGRHPNKAELGMFGVMWSEHCCYKNSRPLLSQFPTEGLRILVGPGENAGVVDLGDGLHLAFKVESHNHPSAVEPFQGAATGVGGILRDIFTMGARPIAILNSLRFGDLEQPRTRRLFHGVVSGISHYGNCVGVPTVGGEVAFDPAYNGNPLVNAMALGLMETDEIVKAGASGIGNPVLYVGSTTGRDGMGGASFASAELSDESLDDRPAVQVGDPFLEKSLIEACLEAFKTGAVVAAQDMG

**That's it!**

The BOFsc for the major macromolecules and the maintenance cost of the cell have been updated in your model using BOFdat. 